In [1]:
import timeit
from sympy import *
import numpy as np
from scipy import linalg, sparse

startTime = timeit.default_timer()
## Problem Statement
stations=8
requests=8
all_variable_count = stations*requests
# Minimum Assignment from Each Station
minimumAssignment = np.random.randint(low = 1, high = 10, size = (requests, stations))
#sparsity_gate=np.round(np.random.random((requests, stations))*1)
sparsity_gate=np.ones((requests, stations))
minimumAssignment=minimumAssignment*sparsity_gate
# Station Count and Request Count
station_counts=5*np.sum(minimumAssignment,axis=0)
request_counts=3*np.sum(minimumAssignment,axis=1)
# Impressions
station_impressions=np.matrix(np.random.uniform(low=10,high=20,size=stations))
request_impressions=3*(minimumAssignment*station_impressions.T).T #Target impressions for request depends on ratio of frames from stations and average station impressions
endTime = timeit.default_timer() - startTime
endTime

0.003157289989758283

In [2]:
startTime = timeit.default_timer()
## Solution
# Cost function
gram_matrix=2*np.kron(station_impressions,station_impressions.T)
Q=np.kron(np.identity(requests,dtype=int),gram_matrix)
p=-2*np.kron(request_impressions,station_impressions)

# Inequality constraints - On stations and on individual non-zero variable
station_G = np.concatenate(np.apply_along_axis(lambda x: linalg.block_diag(*x),1,sparsity_gate)).T
individual_variable_G = -1*np.identity(all_variable_count,dtype=int)*sparsity_gate.reshape(1, all_variable_count)
individual_variable_G = individual_variable_G[np.where(np.sum(individual_variable_G, axis = 1) != 0)[0],]
G = np.concatenate([station_G,individual_variable_G],axis=0)

station_h = station_counts
individual_variable_h = -1*minimumAssignment[np.where(minimumAssignment != 0)]
h = np.concatenate([station_h,individual_variable_h],axis=0)

# Equality constraints - On requests and on individual zero variables
request_A = linalg.block_diag(*sparsity_gate.tolist())
individual_variable_A = np.identity(all_variable_count,dtype=int)*(1 - sparsity_gate.reshape(1, all_variable_count))
individual_variable_A = individual_variable_A[np.where(np.sum(individual_variable_A, axis = 1) != 0)[0],]
A = np.concatenate([request_A,individual_variable_A],axis=0)

request_b = request_counts
individual_variable_b = np.zeros(individual_variable_A.shape[0])
b = np.concatenate([request_b,individual_variable_b],axis=0)
endTime = timeit.default_timer() - startTime
endTime

0.00543354800902307

In [3]:
%%time
startTime = timeit.default_timer()
## Solving optimization
import cvxopt as cv
sol=cv.solvers.coneqp(cv.matrix(Q), cv.matrix(p.T),
                  cv.matrix(G), cv.matrix(h), None,
                  cv.matrix(A), cv.matrix(b))
endTime = timeit.default_timer() - startTime
endTime

     pcost       dcost       gap    pres   dres
 0: -2.1701e+07 -2.1748e+07  5e+04  2e-16  2e-16
 1: -2.1701e+07 -2.1702e+07  5e+02  2e-16  1e-16
 2: -2.1701e+07 -2.1701e+07  5e+00  1e-16  1e-16
Optimal solution found.
CPU times: user 8.44 ms, sys: 8.91 ms, total: 17.4 ms
Wall time: 87.1 ms


In [4]:
optSol = np.round(np.matrix(sol['x']).reshape(requests, stations))
optSol

matrix([[  8.,  19.,   8.,  16.,  12.,  10.,  11.,  16.],
        [  6.,  16.,  10.,   7.,   9.,  13.,  23.,  21.],
        [ 16.,  16.,   8.,  12.,  14.,  12.,  16.,  17.],
        [ 13.,  25.,   8.,  14.,  11.,  16.,  25.,  23.],
        [ 13.,  20.,   9.,  10.,   8.,  20.,  24.,  13.],
        [ 17.,  28.,  16.,  22.,   7.,  11.,   8.,  12.],
        [  7.,  22.,   9.,  11.,   5.,  11.,  16.,  13.],
        [ 10.,  22.,   8.,   9.,   8.,  15.,  18.,  12.]])

In [22]:
station_impressions

matrix([[ 13.80288172,  11.05085169,  14.51930931,  13.58007388,
          14.34456074,  13.76901329,  16.04111783,  11.77482554]])

In [13]:
request_counts

array([ 4647.,  4431.,  4617.,  4806.,  4491.,  4611.,  4371.,  4452.,
        4347.,  4599.,  4356.,  4434.,  4488.,  4224.,  4602.,  4557.,
        4326.,  4554.,  4389.,  4344.,  4320.,  4527.,  4323.,  4233.,
        4695.,  4650.,  4560.,  4551.,  4368.,  4611.,  4473.,  4542.,
        4665.,  4470.,  4470.,  4569.,  4497.,  4461.,  4620.,  4767.,
        4515.,  4683.,  4260.,  4386.,  4620.,  4608.,  4341.,  4245.,
        4695.,  4428.])

In [20]:
minimumAssignment

array([[ 9.,  8.,  2.,  1.,  1.,  6.,  4.,  8.],
       [ 5.,  2.,  1.,  9.,  1.,  5.,  2.,  2.],
       [ 7.,  2.,  4.,  8.,  2.,  9.,  1.,  6.],
       [ 9.,  2.,  7.,  9.,  4.,  2.,  4.,  3.],
       [ 7.,  3.,  7.,  1.,  5.,  5.,  2.,  9.],
       [ 8.,  8.,  5.,  7.,  7.,  2.,  3.,  5.],
       [ 7.,  9.,  6.,  3.,  9.,  6.,  6.,  6.],
       [ 9.,  1.,  9.,  7.,  2.,  2.,  4.,  9.]])

In [15]:
np.sum(optSol, axis = 0)

matrix([[  744.,   888.,   814.,   884.,   759.,   662.,   775.,   555.,
           714.,   705.,   787.,   839.,   764.,   757.,   658.,   672.,
           631.,   707.,   774.,   691.,   859.,   756.,   902.,   843.,
           556.,   630.,   822.,   891.,   831.,   700.,   716.,   875.,
           728.,   750.,   689.,   774.,   745.,   820.,   881.,   626.,
           828.,   721.,   756.,   819.,   874.,   820.,   829.,   850.,
           795.,   794.,   758.,   737.,   756.,   642.,   757.,   465.,
           747.,   903.,   808.,   878.,   727.,   774.,   939.,   745.,
           735.,   640.,   759.,   571.,   658.,   704.,   630.,   747.,
           810.,   643.,   788.,   930.,   554.,   845.,   618.,   714.,
           566.,   574.,   804.,   758.,   810.,   566.,   669.,   909.,
           705.,   691.,   831.,   740.,   655.,   861.,   633.,   760.,
           712.,   728.,   723.,   706.,   642.,   693.,   758.,   895.,
           584.,   836.,   750.,   754.,   792.,   

In [254]:
individual_variable_h

array([ 3.,  7.,  2.,  1.,  7.,  8.,  4.,  7.,  5.,  1.,  3.,  3.,  7.,
        9.,  8.,  1.,  2.,  9.,  5.,  6.,  8.,  3.,  7.,  3.,  2.])

In [353]:
x = np.dot(optSol, station_impressions.T) - np.dot(minimumAssignment, station_impressions.T)*3
y = np.dot(minimumAssignment, station_impressions.T)*3
y

matrix([[ 523.33576933],
        [ 578.51522617]])

In [355]:
np.dot(x.T, x) - np.dot(y.T,y)

matrix([[-608403.72618968]])

In [336]:
sqrt(6.0856e+05)

780.102557360249

In [337]:
(1/2)*optSol.flatten()*Q*optSol.flatten().T + p*optSol.flatten().T

matrix([[-608403.72618968]])

In [5]:
import timeit